# Preprocessing

### Beamforming

In [ ]:
import numpy as np
from obspy.core import read

from infrapy.detection import beamforming_new

if __name__ == '__main__':
    # ######################### #
    #     Define Parameters     #
    # ######################### #
    sac_glob = "data/YJ.BRP*.SAC"
    freq_min, freq_max = 0.5, 2.5
    fk_win_len, window_step = 10.0, 2.5
    sig_start, sig_end = 600, 800
    back_az_vals = np.arange(-180.0, 180.0, 2.0)
    trc_vel_vals = np.arange(300.0, 600.0, 2.5)

    # ######################### #
    #          Run Methods      #
    # ######################### #
    
    # Read data and convert to array format
    x, t, t0, geom = beamforming_new.stream_to_array_data(read(sac_glob))
    M, N = x.shape

    # Define slowness and delays
    slowness = beamforming_new.build_slowness(back_az_vals, trc_vel_vals)
    delays = beamforming_new.compute_delays(geom, slowness)
    
    # Run beamforming in each window and find best beam info
    times, beam_results = [],[]
    for window_start in np.arange(sig_start, sig_end, window_step):
    if window_start + fk_win_len > sig_end:
    break

    X, S, f = beamforming_new.fft_array_data(x, t, window=[window_start, window_start + fk_win_len])
    beam_power = beamforming_new.run(X, S, f, geom, delays, [freq_min, freq_max])
    peaks = beamforming_new.find_peaks(beam_power, back_az_vals, trc_vel_vals)
    times = times + [[t0 + np.timedelta64(int(window_start), 's')]]
    beam_results = beam_results + [[peaks[0][0], peaks[0][1], peaks[0][2] / (1.0 - peaks[0][2]) * (x.shape[0] - 1)]]
    times = np.array(times)[:, 0]
    beam_results = np.array(beam_results)

### Detection

In [ ]:
# Define Parameters 

# Detection params
# times_file, beam_results_file = None, None
times_file, beam_results_file = "data/times.npy", "data/beam_results.npy"

det_win_len = 60 * 5
det_thresh = 0.99
min_seq = 5
TB_prod = 40 * 10
back_az_lim = 10
channel_cnt = 4

if __name__ == '__main__':
    #Load data and prepare analysis 

    if times_file and beam_results_file:
        times = np.load(times_file)
        beam_results = np.load(beam_results_file)
    else:
        print('No beamforming input provided')

    # Run detection analysis 
    # detect_signals(times, beam_results, win_len, TB_prod, channel_cnt, det_p_val=0.99, min_seq=5, back_az_lim=15, fixed_thresh=None, return_thresh=False)

    dets = beamforming_new.detect_signals(times, beam_results, det_win_len, TB_prod, channel_cnt, min_seq=min_seq, back_az_lim=back_az_lim)

    print('\n' + "Detection Summary:")
    for det in dets:
        print("Detection time:", det[0], '\t', "Rel. detection onset:", det[1], '\t',"Rel. detection end:", det[2], '\t',end=' ')
        print("Back azimuth:", np.round(det[3], 2), '\t', "Trace velocity:", np.round(det[4], 2), '\t', "F-stat:", np.round(det[5], 2), '\t', "Array dim:", channel_cnt)

### Association

In [ ]:
from infrapy.association import hjl
from infrapy.utils import data_io
if __name__ == '__main__':
    det_list = data_io.json_to_detection_list('data/example1.dets.json')
    clustering_threshold = 5.0
    
    labels, dists = hjl.run(det_list, clustering_threshold)
    
    clusters, qualities = hjl.summarize_clusters(labels, dists)
for n in range(len(clusters)):
    print("Cluster:", clusters[n], '\t', "Cluster Quality:", 10.0**(qualities[n]))

### Localization

In [ ]:
from infrapy.location import bisl
from infrapy.utils import data_io

if __name__ == '__main__':
    det_list = data_io.json_to_detection_list('data/example2.dets.json')
    
    result,pdf = bisl.run(det_list)
    print(bisl.summarize(result))

### Yield Estimation

In [ ]:
from obspy.core import read
import numpy as np
import matplotlib.pyplot as plt
from infrapy.utils import data_io
from infrapy.propagation import infrasound

from infrapy.characterization import spye
if __name__ == '__main__':
    # ######################### #
    #     Define Parameters     #
    # ######################### #
    det_file = "data/HRR-5.dets.json"
    wvfrm_path = "../infrapy-data/hrr-5/*/*.sac"
    tloss_path = "../infrapy/propagation/priors/tloss/2007_08-"

In [ ]:
ns_opt = "post"
win_buffer = 0.2

src_loc = np.array([33.5377, -106.333961])
freq_band = np.array([0.25, 2.0])
yld_rng = np.array([1.0e3, 1000.0e3])
ref_rng = 1.0

grnd_truth=None
resol = 200

In [ ]:
# Define the detections and spectra

det_list = data_io.json_to_detection_list(det_file)
st_list = [Stream([tr for tr in read(wvfrm_path) if det.station in tr.stats.station]) for det in det_list]
smn_specs = spye.extract_spectra(det_list, st_list, win_buffer=win_buffer, ns_opt=ns_opt)

In [ ]:
# Load TLoss Models

tloss_f_min, tloss_f_max, tloss_f_cnt = 0.025, 2.5, 25
models = [0] * 2
    models[0] = list(np.logspace(np.log10(tloss_f_min), np.log10(tloss_f_max), tloss_f_cnt))
    models[1] = [0] * tloss_f_cnt
    for n in range(tloss_f_cnt):
        models[1][n] = infrasound.TLossModel()
        models[1][n].load(tloss_path + "%.3f" % models[0][n] + "Hz.pri")

In [ ]:
# Run Yield Estimation Methods

yld_results = spye.run(det_list, smn_specs, src_loc, freq_band, models, yld_rng=yld_rng, ref_src_rng=ref_rng, resol=resol)

print('\nResults:')
print('\t' + "Maximum a Posteriori Yield:", yld_results['yld_vals'][np.argmax(yld_results['yld_pdf'])])    print('\t' + "68% Confidence Bounds:", yld_results['conf_bnds'][0])
print('\t' + "95% Confidence Bounds:", yld_results['conf_bnds'][1])

plt.semilogx(yld_results['yld_vals'], yld_results['yld_pdf'])
plt.fill_between(yld_results['yld_vals'], yld_results['yld_pdf'], where=np.logical_and(yld_results['conf_bnds'][0][0] <= yld_results['yld_vals'], yld_results['yld_vals'] <= yld_results['conf_bnds'][0][1]), color='g', alpha=0.25)
plt.fill_between(yld_results['yld_vals'], yld_results['yld_pdf'], where=np.logical_and(yld_results['conf_bnds'][1][0] <= yld_results['yld_vals'], yld_results['yld_vals'] <= yld_results['conf_bnds'][1][1]), color='g', alpha=0.25)

plt.show()
